## La librería fire 

La librería **fire** nos permite generar automáticamente una interfaz de línea
de comando a partir de cualquier objeto python.

### Instalación

Podemos instalar la librería con Pip: 

In [4]:
pip install fire

     |████████████████████████████████| 81 kB 203 kB/s eta 0:00:01
Processing /Users/juan.rodriguezdeleonlocal/Library/Caches/pip/wheels/3f/e3/ec/8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2/termcolor-1.1.0-py3-none-any.whl
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111005 sha256=92b03e75b41aeee145f81abe2cd28b9e72704ee99079c9d6967f004841e9960b
  Stored in directory: /Users/juan.rodriguezdeleonlocal/Library/Caches/pip/wheels/95/38/e1/8b62337a8ecf5728bdc1017e828f253f7a9cf25db999861bec
Successfully built fire
Note: you may need to restart the kernel to use updated packages.


### Ejemplo de uso.

Para usar fire necesitamos instanciar un objeto de la clase `Fire`. Para crear
este objeto le podemos pasar como parámetro cualquier objeto de Python, como
por ejemplo una función:

In [ ]:
# %load 01-hello-world.py
import fire

def hello(name):
    """Saluda al nombre pasado como parámetro.
    """
    return f"Hello {name}!"

if __name__ == '__main__':
    fire.Fire(hello)


Ahora, __desde la línea de comandos__, podemos ejecutar:

    python hello.py
    python hello.py hello john

### Exponer multiples comandos

En el ejemplo anterior solo podíamos acceder a una funcion desde la línea
de comandos. Para poder hacer accesible más funciones tenemos varias opciones:

#### Primera opción 

La forma mas simple de exponer múltiples comandos es escribir multiples
funciones y luego instanciar Fire sin ningun parametro,

In [5]:
# %load 02-simple-calc.py

Observa que Fire ha interpretado correctamente los valores 10 y 20 como números
y no como cadenas de textos. Explicaremos porque pasa esto más adelante.

#### Version 2

Podemos seleccionar las funciones que queremos exponer a la línea de comando,
instanciando de la clase fire pasando como parámetro un diccionario con las
funcines deseadas.

In [11]:
!cat 03-simple-calc.py

import fire

def add(x, y):
    return x + y

def multiply(x, y):
    return x * y

if __name__ == '__main__':
    fire.Fire()


Version 3

La clase Fire también puede ser instanciada con un objeto o una clase. En
ambos casos, todos los métodos definidos en la clase serán accesibles desde la
línea de comandos.

In [13]:
# %load 04-simple-calc.py

En general es preferible pasar como parámetro a la clase Fire otra clase, en
vez de un objeto, porque esto nos permite usar argumentos de la línea de 
comandos para el constructor de la clase.

**Ejercicio**:

Hacer un pequeño programa para imprimir una tabla de multiplicación. El programa debe aceptar un argumento que seria el numero de la tabla que queremos imprimir.

### Acceder a propiedades.

Hasta ahora solo hemos accedido a funciones pero también podemos acceder a las
propiedades de las clases.

En el siguiente ejemplo  veremos un pequeño programa con el que podemos mostrar
información de aeropuertos internacionales usando el código del aeropuerto.

https://github.com/trendct-data/airports.py

In [ ]:
# %load find-airport.py
import csv

import fire


def get_codemap():
    codemap = {}
    with open('iata.csv') as f:
        rd = csv.reader(f, delimiter='\t')
        header = next(rd)
        for items in rd:
            aeropuerto, pais, codigo_iata, resto = items
            codemap[codigo_iata] = (pais, aeropuerto)
    return codemap


class Airport(object):

    def __init__(self, code):
        self.codemap = get_codemap()
        self.code = code.upper()
        self.country = self.codemap[self.code][0]
        self.name = self.codemap[self.code][1]

if __name__ == '__main__':
  fire.Fire(Airport)


### Encadenando llamadas de funciones 

Podemos encadenar llamadas de una forma sencilla, todo lo que tenemos que hacer es escribir una clase cuyo métodos siempre devuelvan `self`, como en el siguiente ejemplo:

In [ ]:
# %load canvas.py
import fire

class BinaryCanvas(object):
    """A canvas with which to make binary art, one bit at a time."""

    def __init__(self, size=10):
        self.pixels = [[0] * size for _ in range(size)]
        self._size = size
        self._row = 0  # The row of the cursor.
        self._col = 0  # The column of the cursor.

    def __str__(self):
        return '\n'.join(' '.join(str(pixel) for pixel in row) for row in self.pixels)

    def show(self):
        print(self)
        return self

    def move(self, row, col):
        self._row = row % self._size
        self._col = col % self._size
        return self

    def on(self):
        return self.set(1)

    def off(self):
        return self.set(0)

    def set(self, value):
        self.pixels[self._row][self._col] = value
        return self

if __name__ == '__main__':
    fire.Fire(BinaryCanvas)


### Salidas personalizada

En el ejemplo anterior hemos pintado en pantalla el resultado de nuestra
órdenes encadenadas con el formato que definimos en el método `__str__`. 

Si se define un método `__str__` propios serán este método el que se usará para
mostrar como salida. Si no se define, se usará la pantalla de ayuda.

### Llamando a funciones y métodos

Los argumentos para los constructores siempre deben pasarse por nombre y usar
la sintaxis `--name=value`.

Los argumentos para otros métodos o funciones se pueden pasar por posición o por nombre.

Una cosa muy útil es que los guiones (`-`) y los subrayados (`_`)  son
intercambiables tanto en los nombres de las funciones como en los argumentos de
la línea de comandos. De igual manera el signo de `=` entre el nombre de la
opción y el valor es opcional.

### Interpretación de los argumentos.

Los tipos de los argumentos vienen determinados por su valor y no por la
asignatura de la funcion o metodo que se vaya a usar. Se puede pasar como
argumento desde la línea de comandos cualquier valor literal que Python puede
interpretar: números, cadenas de textos, tuplas, listas y diccionarios (
dependiendo de la versión de Python que estes usando también conjuntos).

También puedes usar colecciones anidadas siempre y cuando estas solo contengan
literales.

A modo de demostración, el siguiente programa nos dice de qué tipo Python es el
argumento que le pasamos.

In [ ]:
# %load arguments.py
import fire

fire.Fire(lambda obj: type(obj).__name__)


Nota para usuarios de Bash. Ten cuidado con las comillas. Si queremos pasar la
cadena de textos `"10"` en vez del número entero `10` necesitamos escapar las
comillas dos veces porque Bash hara su propia interpretación de la
cadena con lo cual elimina las comillas exteriores.

Las expresiones `True` y `False` se interpretan como valores booleanos. Otra
forma de pasar valores booleanos a nuestro programa sería usar la sintaxis de
doble guión en la forma `-- name` para ajustar el valor a `True` y `--noname`
para ajustar el valor a `False`.

Un *flag* que siempre puedes usar es `--help`, para mostrar explicaciones y
formas de usos. `Fire` incorpora tus *docstrings* dentro de la ayuda que él
genera automáticamente.

Fire tiene varias opciones interesantes más, consúltalas en la página web
https://github.com/google/python-fire si estás interesado.